14-数组的索引参考
====
NumPy Reference ---> Array objects ---> Indexing

可以使用标准的 Python `x [obj]` 语法对 ndarrays 进行检索，其中 x 是数组，obj 是选择。 有三种检索方式可用：
- 字段访问，
- 基本切片，
- 高级索引。

引发哪一个检索方式取决于 `obj`。

**注意：**
在Python中， x[(exp1, exp2, ..., expN)] 等同于 x[exp1, exp2, ..., expN]; 后者只是对前者的语法糖。

# 基本切片和索引

基本切片将Python的切片基本概念扩展到 N 维。当 obj 是：
- 切片对象（由圆括号内的`start:stop:step`符号构造），
- 整数，
- 或切片对象和整数构成的元组时，

会引发“基本切片”式检索。Ellipsis 省略号和 newaxis 对象也可以穿插其中。

使用 N 个整数进行检索的最简单情况是返回表示相应项的数组标量。与在Python中一样，所有索引都是从零开始的：对于第i个索引n_i，有效范围是 `0 <= n_i < d_i`，其中 `d_i` 是数组 shape 的第i个元素。负索引值被解释为从数组的末尾开始计数（即，如果`n_i <0`，则表示`n_i + d_i`）。

通过基本切片生成的所有数组始终是原始数组的视图。

**注：** 

在批处理中>和<为重定向符号，这就意味着不能用 > 来表示大于，< 表示小于，也就意味着不能用 >=、<=、<> 来表示大于等于、小于等于、不等于，还好，在批处理中用了其他的操作符代替它们：

- EQU, 等于
- NEQ, 不等于
- LSS, 小于
- LEQ, 小于或等于
- GTR, 大于
- GEQ, 大于或等于

序列切片的标准规则适用于基于每维的基本切片（包括使用step索引）。要记住的一些有用的概念包括：
- 基本切片语法是 `i：j：k` 其中i是起始索引，j是停止索引，k是步长（`k <> 0`）。 这样就选择了m个元素（在相应的维度中），索引值依次是 `i, i+k, …, i+(m-1)k `，其中 `m = q+r (r<>0)`，q 和 r 分别是`j-i / k`得到的整数商和余数。
`j-i = q*k+r`, 所以 `i+(m-1)k < j`.

In [224]:
# Example
import numpy as np
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
x[1:7:2]

array([1, 3, 5])

- 负i和j被解释为 `n+i` 和 `n+j`，其中 n 是相应维度中的元素数量。负k使得 step 指向更小的索引值。即反向索引。

In [225]:
>>> x[-2:10]

array([8, 9])

In [226]:
>>> x[-3:3:-1]

array([7, 6, 5, 4])

- 假设 n 是要切片的维度中的元素数。如果没有给出 i，则 k>0 时默认为 0，k < 0 时为 n-1。 如果没有给出 j，则对于k> 0，默认为 j=n，对于 k<0，默认为 `j = -n-1`。如果没有给出k，则默认为 k=1。注意 `::` 与 `:` 相同，并且表示在该轴上选择所有索引。

In [227]:
x[:5]

array([0, 1, 2, 3, 4])

- 如果选择元组中的对象数量小于N，则 `:`被假定用于后续所有维度（即后续维度选择全部子元素）：

**注**：对于多维数组（D>2）,`x[i:j:k]`这种基本索引方法，首先在0维度上，根据规则选择行，剩余维度上使用`[:]`这样的索引，即后续维度上索引全部元素。

In [228]:
# Example
>>> x = np.array([[[1],[2],[3]], [[4],[5],[6]]])
>>> x.shape

(2, 3, 1)

In [229]:
x

array([[[1],
        [2],
        [3]],

       [[4],
        [5],
        [6]]])

In [230]:
>>> x[1:2]

array([[[4],
        [5],
        [6]]])

In [231]:
y=np.arange(48)
y = y.reshape(2,3,2,4)
y

array([[[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7]],

        [[ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23]]],


       [[[24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39]],

        [[40, 41, 42, 43],
         [44, 45, 46, 47]]]])

In [232]:
y[1:3]

array([[[[24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39]],

        [[40, 41, 42, 43],
         [44, 45, 46, 47]]]])

- Ellipsis省略号扩展的 `：`对象的数量必需生成与`x.ndim`长度相同的选择元组。可能只存在一个省略号。

In [233]:
 x[...,0]

array([[1, 2, 3],
       [4, 5, 6]])

In [234]:
x.ndim

3

- 选择元组中的每个newaxis对象用于将选择结果的维度扩展一个单位长度维度。添加的维度是 newaxis 对象在选择元组中的位置。

In [235]:
 x[:,np.newaxis,:,:].shape

(2, 1, 3, 1)

- 整数 i 返回与 `i：i + 1`相同的值，除了返回对象的维数减少 1。特别是，具有第 p 个元素的整数（和所有其他`:`条目)的选择元组返回对应的子数组，其维数为 `N - 1`。如果 N = 1，则返回的对象是数组标量。

- 如果选择元组具有所有 `：`条目，除了作为切片对象`i：j：k`的第 p 个条目之外，则返回的数组有 N 个维度，连接通过元素为`i, i+k, …, i + (m - 1) k < j`的整数索引返回的子数组构成。

- 在切片元组中使用多个非`：`条目进行基本切片，就像重复应用单个非`：`条目的切片一样，非`:`条目被连续使用，（所有其他非`：`条目替换为`:`)。 因此，`x [ind1，...，ind2，：]`在基本切片下的行为类似于`x [ind1] [...，ind2，：]`。

**警告**
对于高级索引，上述情况并非如此。

- 可以使用切片来设置数组中的值，但是（与列表不同）永远不会增长数组。要在`x [obj] = value`中设置的值的大小必须（可广播）为与`x [obj]`相同的形状。

**注意**
切片元组总是可以构造为obj并在`x [obj]`表示法中使用。可以在构造中使用切片对象代替`[start：stop：step]`表示法。 例如，`x [1：10：5，:: - 1]`也可以实现为`obj =（slice（1,10,5），slice（None，None，-1））; x [obj]`。 这对于构造适用于任意维数组的通用代码非常有用。

**numpy.newaxis**

newaxis对象可用于所有切片操作，以创建长度为1的轴。 newaxis是'None'的别名，'None'可以代替它使用得到相同的结果。

# 高级检索
下面两种情形触发高级检索：
- 1.选择对象 obj 是非元组序列对象时，（不是元组对象，是其它序列对象，如列表。）
- 2.选择对象 obj 是具有至少一个序列对象或ndarray（数据类型为integer或bool）的元组时。（元组，元组中至少有一个序列对象或ndarray。）

**注意：**判断是否为元组和序列，要对应检索位看。如果选择对象obj是 `x[obj]`的`[ ]`中的内容。

高级检索有两种类型：整数 和 布尔值。

高级索引始终返回数据的副本（与返回视图的基本切片形成对比）。


**警告**
高级索引的定义意味着`x [（1,2,3），]` <span class="burk">（`obj=(1,2,3),`，元组中包含一个序列，触发高级检索。）</span> 与 `x [（1,2,3）]`（<span class="burk">`obj=(1,2,3)`，是元组，但每个元素都是单个值，没有序列或ndarray</span>） 根本不同。后者相当于`x [1,2,3]`，它将触发基本选择，而前者将触发高级索引。 一定要明白为什么会这样。

还要认识到`x [[1,2,3]]`（`obj=[1,2,3]`是一个列表，情形1，非元组序列。）将触发高级索引，而由于上面提到的弃用的数值兼容性，x [[1,2，slice（None）]]将触发基本切片。

**判断是否触发高级检索的步骤：**

obj是否是元组？
1. 不是元组，再判断：
> 是否是序列对象?
    >>是序列对象，触发高级检索；    
    >>不是序列对象，触发基本检索。
2. 是元组，再判断：
> 是否包含至少一个序列元素或 ndarray 元素？
    >> 包含，触发高级检索；    
    >> 不包含，触发基本检索。

## 整数数组检索
整数数组索引允许基于数组的 N-D索引选择数组的任意项，每个整数数组表示该维度的多个索引。

### 纯整数数组检索

当索引值包含与被检索数组的维度数一样的整数数组时，检索是直接了当的，但与切片不同。

高级索引始终作为一个广播和迭代：

高级索引始终作为一个广播和迭代：

`result[i_1, ..., i_M] == x[ind_1[i_1, ..., i_M], ind_2[i_1, ..., i_M],
                           ..., ind_N[i_1, ..., i_M]]`

**注意**，结果形状与（广播）索引数组形状ind_1，...，ind_N相同。

**例**
从每一行开始，应选择一个特定元素。 行索引只是[0,1,2]，列索引指定要为相应行选择的元素，此处为[0,1,0]。 将两者结合使用可以使用高级索引解决任务：

In [236]:
>>> x = np.array([[1, 2], [3, 4], [5, 6]])
x

array([[1, 2],
       [3, 4],
       [5, 6]])

In [237]:
>>> x[[0, 1, 2], [0, 1, 0]]

array([1, 4, 5])

为了实现类似于上面的基本切片的行为，可以使用广播。 函数`ix_`可以帮助这个广播。通过示例可以最好地理解这一点。

**例**
从 4x3 数组中选择角元素，应使用高级检索。因此，需要选择列为[0,2]之一且行为[0,3]之一的所有元素。要使用高级检索，需要显式地选择所有元素。使用前面解释的方法可以这样写：

In [238]:
>>> x = np.array([[ 0,  1,  2],
...            [ 3,  4,  5],
...            [ 6,  7,  8],
...            [ 9, 10, 11]])
x

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [239]:
>>> rows = np.array([[0, 0],
                     [3, 3]], dtype=np.intp)

In [240]:
>>> columns = np.array([[0, 2],
                        [0, 2]], dtype=np.intp)

In [241]:
>>> x[rows, columns]

array([[ 0,  2],
       [ 9, 11]])

但是，由于上面的索引数组只是重复自身，因此可以使用广播（比较诸如`rows [：，np.newaxis] + columns`之类的操作）来简化：

In [242]:
>>> rows = np.array([0, 3], dtype=np.intp)

In [243]:
>>> columns = np.array([0, 2], dtype=np.intp)

In [244]:
>>> rows[:, np.newaxis]

array([[0],
       [3]], dtype=int64)

In [245]:
>>> x[rows[:, np.newaxis], columns]

array([[ 0,  2],
       [ 9, 11]])

这个广播也可以使用`ix_`函数来实现：

In [246]:
x[np.ix_(rows, columns)]

array([[ 0,  2],
       [ 9, 11]])

请注意，如果没有np.ix_调用，则只会选择对角线元素，如上例所示。 对于使用多个高级索引进行索引，这个差异是最重要的。

# 结合高级和基本检索

当索引中至少有一个切片（:)，省略号（...）或np.newaxis（或者数组的维度多于高级索引）时，行为可能会更复杂。这就像连接每个高级索引元素的索引结果一样

在最简单的情况下，只有一个高级索引。一个单一的高级索引可以替代一个切片，结果数组也会是一样的，但是，它是一个副本，可能有不同的内存布局。在可能的情况下，切片是更好的选择。

**注意** 参考触发高级索引的两种情形。

In [247]:
>>> x[1:2, 1:3]  # obj 是元组，但其中没有序列元素，元组的元素是两个切片。

array([[4, 5]])

In [248]:
>>> x[1:2, [1, 2]] # obj 是元组，但第二个元素是一个列表序列，触发高级检索。

array([[4, 5]])

理解这种情况的最简单方法可能是根据结果的 shape 进行思考。 索引操作分为两部分，即由基本索引（不包括整数）定义的子空间和来自高级索引部分的子空间。 需要区分两种索引组合：

- 高级索引由切片，省略号或 newaxis 分隔。 例如`x [arr1，：，arr2]`。
- 高级索引彼此相邻。 例如`x [...，arr1，arr2，：]`但不是`x [arr1，：，1]`，因为在这点上， `1` 是一个高级索引。

在第一种情况下，高级索引操作产生的维度首先出现在结果数组中，然后是子空间维度。 

在第二种情况下，高级索引操作的维度将插入到结果数组中与原始被检索数组中相同的位置（后一种逻辑使简单的高级索引行为就像切片一样）。

**示例：**

假设`x.shape`是`（10,20,30`）而`ind`是形状为`（2,3,4）`的检索用`intp`数组，那么`result = x [...，ind，：]`有形状`（10,2,3,4,30）`，因为（20，）形状的子空间已被（2,3,4）形形状的广播索引子空间所取代。 如果我们让`i，j，k`在`（2,3,4）`形状子空间上循环，那么结果`[...，i，j，k，：] = x [...，ind [i，j，k],:]`。 此示例产生与x.take（ind，axis = -2）相同的结果。

In [249]:
x = np.arange(6000,dtype="intp")
x = x.reshape(10,20,30)

**说明：**官方文档中说x的shape=(10,20,30)，ind的形状是（2，3，4）的索引数组，由于ind元素的个数是`2*3*4=24`个。会触发‘ cannot reshape array of size 20 into shape (2,3,4)’错误。索引数组ind元素的个数不能超过对应维度的值。

In [250]:
ind = np.arange(18,dtype='intp').reshape(2,3,3)
ind

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]]], dtype=int64)

In [251]:
result = x[...,ind,:] #混合检索的第二种情况，ind检索出的结果插入到对应位置，这里是 1 维度上，替换了20.
result.shape

(10, 2, 3, 3, 30)

In [252]:
result = x.take(ind, axis=-2)
result.shape

(10, 2, 3, 3, 30)

**示例**
设x.shape为`（10,20,30,40,50）`，并假设ind_1和ind_2可以广播到形状（2,3,4）。 然后`x [：，ind_1，ind_2]`具有形状（10,2,3,4,40,50），因为来自X的（20,30）形子空间已被替换为来自索引的（2,3,4）子空间。 但是，`x [：，ind_1，：，ind_2]`具有形状（2,3,4,10,30,50），因为在索引子空间中没有明确的位置，因此它被添加到开头（<span class="burk">混合检索的第一种情形。</span>）。 始终可以使用`.transpose（）`在任何需要的位置移动子空间。 请注意，此示例无法使用 take 代替。

# 布尔数组检索

当obj是布尔类型的数组对象时，会发生高级检索，例如可能从比较运算符返回。单个布尔索引数组实际上与x `[obj.nonzero（）]`相同，其中，如上所述，`obj.nonzero（）`返回一个整数索引数组的元组（长度为obj.ndim），显示obj的True元素。 但是，当`obj.shape == x.shape`时它会更快。

如果`obj.ndim == x.ndim`，则`x [obj]`返回一个1维数组，该数组是 obj 的 True 值替换为 x 中对应位置元素值。 搜索顺序为行主，C风格。 如果 obj 在 x 的边界之外的条目处具有True值，则将引发索引错误。 如果 obj 小于 x，则与填充False相同。

**示例**
一个常见的用例是过滤所需的元素值。例如，可能希望从数组中选择非NaN的所有条目：

In [253]:
>>> x = np.array([[1., 2.], [np.nan, 3.], [np.nan, np.nan]])
>>> x[~np.isnan(x)]

array([ 1.,  2.,  3.])

或者希望给所有的负值加一个常量：

In [254]:
>>> x = np.array([1., -1., -2., 3])
>>> x[x < 0] += 20
>>> x

array([  1.,  19.,  18.,   3.])

通常，如果索引包括布尔数组，则结果与将`obj.nonzero（）`插入相同位置并使用上述整数数组索引机制相同。

` x [ind_1，boolean_array，ind_2]`等价于`x [（ind_1，）+ boolean_array.nonzero（）+（ind_2，）]`。

如果只有一个布尔数组且没有整数索引数组，则是直截了当的。 必须注意确保布尔索引具有与其应该使用的维度完全相同的维度。

**示例**
从一个数组中，选择所有的行，它们的总和小于或等于2：

In [255]:
>>> x = np.array([[0, 1], [1, 1], [2, 2]])
>>> rowsum = x.sum(-1)
>>> x[rowsum <= 2, :]

array([[0, 1],
       [1, 1]])

但是，如果rowsum也有两个维度：

In [256]:
x.shape

(3, 2)

In [257]:
>>> rowsum = x.sum(-1, keepdims=True)
>>> rowsum.shape
# rowsum = np.arange(6).reshape(3,2)

(3, 1)

In [258]:
rowsum

array([[1],
       [2],
       [4]])

In [259]:
#>>> x[rowsum<=2,:]    # fails

In [260]:
#>>> x[rowsum<=2]

由于额外的维度，最后一个(`x[rowsum<=2]`)只给出了第一个元素。 比较rowsum.nonzero（）以了解此示例。

<span class="burk">**注**：该段是官方文档的示例，但返回错误。</span>

使用`obj.nonzero（）`进行类比可以最好地理解组合多个布尔索引数组或布尔与整数索引数组。 函数`ix_`也支持布尔数组，并且可以毫无意外地工作。

**示例**

使用布尔索引选择加起来为偶数的所有行。 同时，使用高级整数索引选择列0和2。 使用ix_函数可以通过以下方式完成：

In [261]:
>>> x = np.array([[ 0,  1,  2],
               [ 3,  4,  5],
               [ 6,  7,  8],
               [ 9, 10, 11]])
x

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [262]:
>>> rows = (x.sum(-1) % 2) == 0
>>> rows

array([False,  True, False,  True], dtype=bool)

In [263]:
>>> columns = [0, 2]
>>> x[np.ix_(rows, columns)]

array([[ 3,  5],
       [ 9, 11]])

没有`np.ix_`调用或只选择对角线元素。

或者没有`np.ix_`（比较整数数组的例子）：

In [264]:
>>> rows = rows.nonzero()[0]
rows

array([1, 3], dtype=int64)

In [265]:
>>> x[rows[:, np.newaxis], columns]

array([[ 3,  5],
       [ 9, 11]])

# 细节说明
Detailed notes

这些是一些详细的注释，对于日常索引（无特定顺序）并不重要：

- NumPy原生索引类型是intp，可能与默认的整数数组类型不同。 intp是足以安全索引任何数组的最小数据类型; 对于高级索引，它可能比其他类型更快。
- 对于高级赋值，通常不保证迭代顺序。 这意味着如果元素被设置多次，则无法预测最终结果。

- 空（元组）索引是零维数组的完整标量索引。 如果x是零维，则`x [（）]`返回标量，否则返回视图。 另一方面，`x [...]`总是返回一个视图。
- 如果索引中存在零维数组并且它是完整的整数索引，则结果将是标量而不是零维数组。 （不会触发高级索引。）

- 当省略号（...）存在但没有大小（即替换为零）时，结果仍将始终为数组。 如果没有高级索引，则为视图，否则为副本。
- 布尔数组的nonzero等价不适用于零维布尔数组。

- 当高级索引操作的结果没有元素但单个索引超出范围时，是否引发IndexError是未定义的（例如`x [[]，[123]]`，其中123超出界限）。
- 当在赋值期间发生转换错误时（例如，使用字符串序列更新数值数组），被分配的数组可能最终处于不可预测的部分更新状态。 但是，如果发生任何其他错误（例如超出范围索引），则数组将保持不变。

- 高级索引结果的内存布局针对每个索引操作进行了优化，并且不能假设特定的内存顺序。
- 当使用子类（尤其是操纵其形状的子类）时，默认的`ndarray .__ setitem__`行为将调用`__getitem__`进行基本索引，但不调用高级索引。对于这样的子类，最好使用数据的基类ndarray视图调用`ndarray .__ setitem__`。 如果子类`__getitem__`没有返回视图，则必须执行此操作。

# 字段访问

Field Access

如果ndarray对象是结构化数组，则可以通过使用字符串索引数组来访问数组的字段，类似于字典。

索引` x ['field-name']`返回一个数组的新视图，该视图与x的形状相同（当字段是子数组时除外）但数据类型为`x.dtype ['field-name']`并且仅包含指定字段中的部分数据。记录数组标量也可以这种方式“索引”。

索引到结构化数组也可以使用字段名称列表来完成，例如，` x[['field-name1','field-name2']]`。目前，这将返回一个新数组，其中包含列表中指定的字段中的值的副本。从NumPy 1.7开始，不推荐返回副本以支持返回视图。现在将继续返回副本，但在写入副本时将发出FutureWarning。如果依赖于当前行为，那么我们建议显式复制返回的数组，即使用`x [['field-name1'，'field-name2']].copy（）`。这将适用于NumPy的过去和未来版本。

如果访问的字段是子数组，则子数组的维度将附加到结果的形状。

**示例**：

In [271]:
>>> x = np.zeros((2,2), dtype=[('a', np.int32), ('b', np.float64, (3,3))])

In [272]:
>>> x['a'].shape

(2, 2)

In [273]:
>>> x['a'].dtype

dtype('int32')

In [274]:
>>> x['b'].shape

(2, 2, 3, 3)

In [275]:
>>> x['b'].dtype

dtype('float64')

# 扁平的迭代器检索
Flat Iterator indexing

`x.flat`返回一个迭代器，它将迭代整个数组（以C连续样式，最后一个索引变化最快）。 只要选择对象不是元组，也可以使用基本切片或高级索引对此迭代器对象建立索引。从`x.flat`是一维视图的事实可以清楚地看出这一点。 它可用于具有1维C风格平面索引的整数索引。 因此，任何返回数组的形状都是整数索引对象的形状。